In [1]:
import pandas as pd
import numpy as np
import sqlite3

# conn = sqlite3.connect('C:/Users/Mark Rozenberg/Yellow-Taxi-Climate/Data/Main_DB.db')
# pd.read_csv('C:/Users/Mark Rozenberg/Yellow-Taxi-Climate/2023_Yellow_Taxi_Trip_Data.csv').to_sql('dt_taxi_23', conn, if_exists='replace', index = False)

In [5]:
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-21"
os.environ["SPARK_HOME"] = "C:\\spark"
os.environ['HADOOP_HOME'] = "C:\\hadoop"
# os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:\\hadoop\\lib\\native"
# os.environ['HADOOP_OPTS'] = "-Djava.library.path=C:\\hadoop\\lib\\native"

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Yellow Taxi Data") \
    .master("local[*]") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
# Read the CSV file
df_spark = spark.read.csv('C:/Users/Mark Rozenberg/Yellow-Taxi-Climate/Data/2023_Yellow_Taxi_Trip_Data.csv', header=True, inferSchema=True)
print(df_spark.count())

38310226


In [3]:
38310226/1000000

38.310226

In [5]:
print(len(list(df_spark.columns)))

19


In [17]:
import psycopg2

# Connect to your postgres DB
conn = psycopg2.connect(
    dbname='Yellow-Taxi',
    user='postgres',
    password='7466239',
    host='localhost',
    port='5432'
)

# Open a cursor to perform database operations
cur = conn.cursor()

In [18]:
cur.execute(''' drop table if exists taxi_data''')
conn.commit()

In [15]:
pd.read_csv('C:/Users/Mark Rozenberg/Yellow-Taxi-Climate/Data/2023_Yellow_Taxi_Trip_Data.csv', nrows=1).columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [19]:
# Create table in PostgreSQL
cur.execute('''
    CREATE TABLE IF NOT EXISTS taxi_data (
        VendorID INT,
        tpep_pickup_datetime TIMESTAMP,
        tpep_dropoff_datetime TIMESTAMP,
        passenger_count INT,
        trip_distance FLOAT,
        RatecodeID INT,
        store_and_fwd_flag CHAR(1),
        PULocationID INT,
        DOLocationID INT,
        payment_type INT,
        fare_amount FLOAT,
        extra FLOAT,
        mta_tax FLOAT,
        tip_amount FLOAT,
        tolls_amount FLOAT,
        improvement_surcharge FLOAT,
        total_amount FLOAT,
        congestion_surcharge FLOAT,
        airport_fee FLOAT
    )
''')
conn.commit()

In [20]:
cur.execute('SELECT COUNT(*) FROM taxi_data')
row_count = cur.fetchone()[0]
print(row_count)

77966324


In [21]:
# Read data from PostgreSQL table into a pandas DataFrame
taxi_data_df = pd.read_sql_query('SELECT * FROM taxi_data limit 5', conn)
taxi_data_df

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_16760\98094685.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  taxi_data_df = pd.read_sql_query('SELECT * FROM taxi_data limit 5', conn)


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2022-07-19 22:12:14,2022-07-19 22:38:09,1,13.79,1,N,87,127,1,39.5,0.5,0.5,8.66,0.0,0.3,51.96,2.5,0.0
1,2,2022-07-19 22:55:56,2022-07-19 23:04:33,1,1.77,1,N,75,238,2,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5,0.0
2,2,2022-07-19 22:07:09,2022-07-19 22:30:26,2,4.50,1,N,238,234,1,18.5,0.5,0.5,0.00,0.0,0.3,22.30,2.5,0.0
3,2,2022-07-19 22:32:29,2022-07-19 22:49:48,2,3.89,1,N,234,263,1,14.5,0.5,0.5,2.00,0.0,0.3,20.30,2.5,0.0
4,1,2022-07-19 22:06:40,2022-07-19 22:27:06,1,4.90,1,N,141,112,2,18.5,3.0,0.5,0.00,0.0,0.3,22.30,2.5,0.0
